## TP3 - Exercice 2 - 17/10/2025

### Objectif :

- Appliquer l'algorithme de **groupement K-Means**, de l'implémentation manuelle à l'utilisation de bibliothèques Python, afin de comparer les résultats, analyser la compacité et la séparation des clusters, et évaluer les performances selon différents choix de distance.



1. Implémenter le code qui décrit l’exercice 1 et vérifier par simulation la conformité des
résultats des calculs manuels.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin

In [2]:
# Données initiales encodées
data = np.array([
    [1.0, 0.0, 0.0, 0.3],
    [0.3, 0.3, 0.75, 0.0],
    [0.0, 1.0, 0.0, 0.3],
    [0.75, 0.3, 0.0, 0.3],
    [0.0, 1.0, 0.0, 0.3],
    [0.0, 0.0, 1.0, 0.3]
])

columns = ['x1 (café)', 'x2 (courses)', 'x3 (sport)', 'x4 (maison)']
df = pd.DataFrame(data, columns=columns)
df.index = np.arange(1, len(df)+1)
df


,x1 (café),x2 (courses),x3 (sport),x4 (maison)
1,1.00,0.0,0.00,0.3
2,0.30,0.3,0.75,0.0
3,0.00,1.0,0.00,0.3
4,0.75,0.3,0.00,0.3
5,0.00,1.0,0.00,0.3
6,0.00,0.0,1.00,0.3


Définition des centroïdes initiaux G₁ et G₂

In [17]:
# Centroïdes initiaux
G1 = np.array([0.3, 0.1, 0.2, 0.6])
G2 = np.array([0.2, 0.2, 0.6, 0.5])

centroids = np.vstack([G1, G2])
centroids_df = pd.DataFrame(centroids, columns=columns, index=['G1', 'G2'])
centroids_df


,x1 (café),x2 (courses),x3 (sport),x4 (maison)
G1,0.3,0.1,0.2,0.6
G2,0.2,0.2,0.6,0.5


Attribution des points aux centroïdes les plus proches (distance euclidienne)

In [18]:
# Calcul de la distance et attribution
labels = pairwise_distances_argmin(df.values, centroids)

# Affichage de l’appartenance de chaque individu
assignments = pd.DataFrame({
    'Personne': df.index,
    'Cluster assigné': ['G1' if l == 0 else 'G2' for l in labels]
})
assignments

,Personne,Cluster assigné
0,1,G1
1,2,G2
2,3,G1
3,4,G1
4,5,G1
5,6,G2


Calcul des nouveaux centroïdes (après 1 itération)

In [19]:
# Calcul des nouveaux centroïdes
new_centroids = np.array([
    df.values[labels == 0].mean(axis=0),
    df.values[labels == 1].mean(axis=0)
])

new_centroids_df = pd.DataFrame(new_centroids, columns=columns, index=['G1\'', 'G2\''])
new_centroids_df


,x1 (café),x2 (courses),x3 (sport),x4 (maison)
G1',0.4375,0.575,0.000,0.30
G2',0.1500,0.150,0.875,0.15


On doit passer à l'itération 2 car les centroides ne sont pas stables

Calcul des nouveaux centroïdes (après 2 itérations)

In [20]:
# -------------------------------
# 2️⃣ DEUXIÈME ITÉRATION
# -------------------------------

# Étape 1 : Réattribution des individus selon les nouveaux centroïdes
labels_iter2 = pairwise_distances_argmin(df.values, new_centroids)

# Étape 2 : Affichage du nouveau groupe de chaque individu
assignments_iter2 = pd.DataFrame({
    'Personne': df.index,
    'Cluster assigné (itération 2)': ['G1' if l == 0 else 'G2' for l in labels_iter2]
})
assignments_iter2

,Personne,Cluster assigné (itération 2)
0,1,G1
1,2,G2
2,3,G1
3,4,G1
4,5,G1
5,6,G2


In [21]:
# Étape 3 : Calcul des nouveaux centroïdes
centroids_iter2 = np.array([
    df.values[labels_iter2 == 0].mean(axis=0),
    df.values[labels_iter2 == 1].mean(axis=0)
])

centroids_iter2_df = pd.DataFrame(
    centroids_iter2,
    columns=columns,
    index=["G1''", "G2''"]
)
centroids_iter2_df


,x1 (café),x2 (courses),x3 (sport),x4 (maison)
G1'',0.4375,0.575,0.000,0.30
G2'',0.1500,0.150,0.875,0.15


##### 2éme vérification avec la bibliothèque Kmeans de sklearn 

Dans cette partie, on applique directement l’algorithme K-Means fourni par la bibliothèque **scikit-learn** pour vérifier la conformité des résultats obtenus manuellement.

##### Itération 1 — Initialisation avec G1 et G2
On exécute une première itération à partir des centroïdes donnés.


In [22]:
# Centroïdes initiaux
G1 = np.array([0.3, 0.1, 0.2, 0.6])
G2 = np.array([0.2, 0.2, 0.6, 0.5])
centroids_init = np.vstack([G1, G2])

# Première itération
kmeans_iter1 = KMeans(n_clusters=2, init=centroids_init, n_init=1, max_iter=1, random_state=42)
kmeans_iter1.fit(df.values)

# Résultats itération 1
assignments_iter1 = pd.DataFrame({
    'Personne': df.index,
    'Cluster (Itération 1)': ['G1' if l == 0 else 'G2' for l in kmeans_iter1.labels_]
})
centroids_iter1 = pd.DataFrame(kmeans_iter1.cluster_centers_, columns=columns, index=['G1′', 'G2′'])

assignments_iter1, centroids_iter1


(   Personne Cluster (Itération 1)
 0         1                    G1
 1         2                    G2
 2         3                    G1
 3         4                    G1
 4         5                    G1
 5         6                    G2,
      x1 (café)  x2 (courses)  x3 (sport)  x4 (maison)
 G1′     0.4375         0.575       0.000         0.30
 G2′     0.1500         0.150       0.875         0.15)

##### Itération 2 — À partir des centroïdes obtenus
On continue l’algorithme à partir des centroïdes calculés à l’itération 1.


In [23]:
# Deuxième itération à partir des centroïdes trouvés
kmeans_iter2 = KMeans(n_clusters=2, init=kmeans_iter1.cluster_centers_, n_init=1, max_iter=1, random_state=42)
kmeans_iter2.fit(df.values)

# Résultats itération 2
assignments_iter2 = pd.DataFrame({
    'Personne': df.index,
    'Cluster (Itération 2)': ['G1' if l == 0 else 'G2' for l in kmeans_iter2.labels_]
})
centroids_iter2 = pd.DataFrame(kmeans_iter2.cluster_centers_, columns=columns, index=['G1″', 'G2″'])

assignments_iter2, centroids_iter2


(   Personne Cluster (Itération 2)
 0         1                    G1
 1         2                    G2
 2         3                    G1
 3         4                    G1
 4         5                    G1
 5         6                    G2,
      x1 (café)  x2 (courses)  x3 (sport)  x4 (maison)
 G1″     0.4375         0.575       0.000         0.30
 G2″     0.1500         0.150       0.875         0.15)

##### Observation :

Les résultats obtenus à partir des trois approches — **calcul manuel en classe**, **implémentation “hard-coded” en Python**, et **application de la bibliothèque `KMeans` de scikit-learn** —  
sont **identiques** en termes de :

- **Appartenance des individus** aux groupes (G1 et G2)
- **Valeurs finales des centroïdes**

✅ Cela confirme la **conformité de l’implémentation manuelle** avec la version standard de l’algorithme K-Means.  
L’algorithme converge vers les **mêmes centroïdes optimaux** quelle que soit la méthode utilisée, ce qui valide la justesse des calculs réalisés.


2. Donner quelques bases de données utilisées dans le cas du groupement pour exécuter des
benchmark

### 🗂️ Bases de données utilisées pour les benchmarks du groupement (K-Means)

Dans le cadre de l’évaluation et du benchmarking des algorithmes de **groupement (clustering)** comme **K-Means**, plusieurs bases de données standard sont utilisées dans la recherche et l’enseignement.  
Elles permettent de tester la **qualité du partitionnement**, la **convergence** et la **performance** de l’algorithme.

#### 🔹 1. **Iris Dataset (Fisher, 1936)**
- L’une des bases les plus utilisées pour tester K-Means.  
- Contient **150 fleurs** décrites par **4 attributs** (longueur et largeur des sépales et pétales).  
- Objectif : regrouper les fleurs selon leur espèce (`setosa`, `versicolor`, `virginica`).

#### 🔹 2. **Customer Segmentation Dataset (Marketing)**
- Données clients (âge, revenu, score d’achat, etc.).  
- Utilisée pour des applications réelles de segmentation de marché.

#### 🔹 3. **Mall Customers Dataset**
- Jeu de données simulant les clients d’un centre commercial.  
- Très utilisé pour illustrer les clusters visuellement (Revenu vs Score de dépense).

Ces trois bases offrent un **bon équilibre entre données académiques et données réelles**, permettant de **tester K-Means sur différents contextes**.


3. Donner la bibliothèque et les fonctions nécessaires sur python qui permettent d’appliquer
l’algorithme K means. 


- **Bibliothèque** : `scikit-learn`  
- **Classe principale** : `sklearn.cluster.KMeans`  

**Exemples :**  
```python
kmeans = KMeans(n_clusters=k, init='k-means++', n_init=10, max_iter=300, random_state=42)
kmeans.fit(X)             # entraîner le modèle
kmeans.predict(X)         # prédire les clusters
kmeans.fit_predict(X)     # fit + predict
kmeans.cluster_centers_   # centroïdes finaux
kmeans.labels_            # étiquettes des clusters


4. Donner les hyper paramètres de l’algorithme K means. 


- Nombre de clusters (n_clusters) : nombre de groupes à former.
- Nombre d’itérations (max_iter) : nombre maximal d’itérations pour ajuster les centroïdes.
- Valeur initiale des centroïdes (init) : méthode d’initialisation.

5. Donner une évaluation de cet algorithme en se basant sur les notions de dispersion. Dresser
un tableau des résultats des performances selon le choix de la distance. 

#### Évaluation de K-Means par dispersion

- **Dispersion intra-classe (W)** : mesure la compacité des clusters  
$$
W = \sum_{k=1}^{K} \sum_{x_i \in C_k} ||x_i - \mu_k||^2
$$  
> Plus W est faible → clusters plus compacts.

- **Dispersion inter-classe (B)** : mesure la séparation entre clusters  
$$
B = \sum_{k=1}^{K} n_k ||\mu_k - \mu||^2
$$  
> Plus B est grand → clusters mieux séparés.

- **Rapport B/W** : évaluation globale  
$$
\text{Indice} = \frac{B}{W}
$$  
> Plus le ratio est élevé → meilleur groupement (forte séparation + bonne compacité).

**Résumé** : un bon K-Means → W faible, B élevé, B/W élevé.


In [31]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# -------------------------------
# Données
# -------------------------------
X = np.array([
    [1.0, 0.0, 0.0, 0.3],
    [0.3, 0.3, 0.75, 0.0],
    [0.0, 1.0, 0.0, 0.3],
    [0.75, 0.3, 0.0, 0.3],
    [0.0, 1.0, 0.0, 0.3],
    [0.0, 0.0, 1.0, 0.3]
])

# -------------------------------
# Fonction dispersion
# -------------------------------
def dispersion_metrics(X, labels, centroids):
    W = 0
    K = centroids.shape[0]
    overall_mean = X.mean(axis=0)
    for k in range(K):
        cluster_points = X[labels == k]
        W += np.sum(np.linalg.norm(cluster_points - centroids[k], axis=1)**2)
    B = 0
    for k in range(K):
        n_k = np.sum(labels == k)
        B += n_k * np.linalg.norm(centroids[k] - overall_mean)**2
    ratio = B / W if W != 0 else np.inf
    return W, B, ratio

# -------------------------------
# 1️⃣ Euclidean avec KMeans
# -------------------------------
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)
labels_euc = kmeans.labels_
centroids_euc = kmeans.cluster_centers_
W_euc, B_euc, ratio_euc = dispersion_metrics(X, labels_euc, centroids_euc)

# -------------------------------
# 2️⃣ Manhattan (calcul manuel)
# -------------------------------
centroids_man = X[np.random.choice(len(X), 2, replace=False)]
for _ in range(10):
    distances = np.sum(np.abs(X[:, None, :] - centroids_man[None, :, :]), axis=2)
    labels_man = np.argmin(distances, axis=1)
    for k in range(2):
        if np.any(labels_man == k):
            centroids_man[k] = X[labels_man == k].mean(axis=0)
W_man, B_man, ratio_man = dispersion_metrics(X, labels_man, centroids_man)

# -------------------------------
# Tableau comparatif
# -------------------------------
results_df = pd.DataFrame({
    'Distance': ['Euclidean','Manhattan'],
    'W (Intra)': [round(W_euc,3), round(W_man,3)],
    'B (Inter)': [round(B_euc,3), round(B_man,3)],
    'B/W (Indice)': [round(ratio_euc,3), round(ratio_man,3)]
})

print(results_df)


    Distance  W (Intra)  B (Inter)  B/W (Indice)
0  Euclidean      1.731      1.402         0.810
1  Manhattan      1.776      1.357         0.764


Déduction

**1. Compacité des clusters (W) :**

- **Euclidean** → W = 1.731 (plus faible)  
- **Manhattan** → W = 1.776 (plus élevé)  

> Les clusters obtenus avec **Euclidean** sont légèrement plus compacts.

**2. Séparation des clusters (B) :**

- **Euclidean** → B = 1.402 (plus élevé)  
- **Manhattan** → B = 1.357 (moins élevé)  

> Les clusters Euclidean sont mieux séparés les uns des autres.

**3. Indice global B/W :**

- **Euclidean** = 0.810 > **Manhattan** = 0.764  

> Le compromis compacité/séparation est meilleur avec **Euclidean**.

**Conclusion :**  
Pour ce dataset, l’algorithme **K-Means** fonctionne mieux avec la distance **Euclidean**, car il produit des clusters à la fois plus compacts et mieux séparés que Manhattan.
